# AES workflow

當使用AES（高級加密標準）加密算法時，通常遵循以下工作流程：

## AES加密工作流程

1. **密鑰擴展（Key Expansion）**：
   - 使用初始密鑰，通過密鑰擴展算法生成一系列輪密鑰（Round Keys），以供後續的輪密鑰加操作使用。

2. **初始輪密鑰加（Initial Round Key Addition）**：
   - 使用初始輪密鑰，將明文與初始輪密鑰進行按位異或操作（XOR）。

3. **多輪加密（Multiple Rounds Encryption）**：
   - 重覆執行以下步驟：
     - **字節代替（SubBytes）**：使用S盒（Substitution Box）替換每個字節，增加非線性性。
     - **行移位（ShiftRows）**：按照規定的規則對每一行進行循環左移，實現行內數據混淆。
     - **列混淆（MixColumns）**：將每一列視為多項式，與一個固定的多項式進行乘法運算，增加數據的擴散性。
     - **輪密鑰加（Round Key Addition）**：使用當前輪密鑰與狀態矩陣進行按位異或操作（XOR）。

4. **最終加密輪**：
   - 在最後一輪執行字節代替、行移位和輪密鑰加操作，但不執行列混淆操作。

5. **輸出加密結果**：
   - 將加密後的狀態矩陣轉換為一維數組，並輸出為密文。

## AES解密工作流程

1. **密鑰擴展（Key Expansion）**：
   - 同樣使用初始密鑰，通過密鑰擴展算法生成一系列輪密鑰。

2. **初始輪密鑰加（Initial Round Key Addition）**：
   - 使用初始輪密鑰，將密文與初始輪密鑰進行按位異或操作（XOR）。

3. **多輪解密（Multiple Rounds Decryption）**：
   - 重覆執行以下步驟（與加密步驟相反執行）：
     - **逆行移位（Inverse ShiftRows）**：按照逆規則對每一行進行循環右移，與加密過程相對應。
     - **逆字節代替（Inverse SubBytes）**：使用逆S盒替換每個字節，與加密過程相對應。
     - **輪密鑰加（Round Key Addition）**：使用當前輪密鑰與狀態矩陣進行按位異或操作（XOR）。

4. **最終解密輪**：
   - 在最後一輪執行逆字節代替、逆行移位和輪密鑰加操作，但不執行列混淆操作。

5. **輸出解密結果**：
   - 將解密後的狀態矩陣轉換為一維數組，並輸出為明文。

## 密鑰長度和輪數

- AES支持多種密鑰長度：128位、192位和256位，對應的輪數分別為10輪、12輪和14輪。
- 密鑰長度決定了加密算法的安全性和強度，而輪數決定了加密過程的覆雜度和疊代次數。

這就是AES加密算法的工作流程，涵蓋了加密和解密過程。


# Example of AES encryption

In [13]:
def encrypt(self, plaintext: list[int]) -> list[int]:
    state = list(plaintext)
    state = self._add_round_key(state, self.round_keys[0])
    for round_idx in range(1, 11):
        state = self._sub_bytes(state)
        state = self._shift_rows(state)
        if round_idx < 10:
            state = self._mix_columns(state)
        state = self._add_round_key(state, self.round_keys[round_idx])
    return state


@classmethod
def _rot_bytes(cls, state: list[int]) -> list[int]:
    return state[1:] + [state[0]]


@classmethod
def generate_round_keys(cls, state: list[int]) -> list[int]:
    round_keys: list[list[int]] = [list(state)]
    for round_idx in range(10):
        round_state: list[int] = cls._sub_bytes(cls._rot_bytes(state[-4:]))
        round_state[0] = round_state[0] ^ cls.rcon[round_idx]
        for idx in range(16):
            state[idx] = state[idx] ^ round_state[idx % 4]
            if idx % 4 == 3:
                round_state = state[idx - 3 : idx + 1]
        round_keys.append(list(state))
    return round_keys


@classmethod
def _add_round_key(cls, state: list[int], round_key: list[int]) -> list[int]:
    for i in range(16):
        state[i] = state[i] ^ round_key[i]
    return state


# fmt: off
s_box: tuple[int] = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16,
)


# fmt: on
@classmethod
def _sub_bytes(cls, state: list[int]) -> list[int]:
    return [cls.s_box[byte] for byte in state]


@classmethod
def _shift_rows(cls, state: list[int]) -> list[int]:
    state[1], state[5], state[9], state[13] = state[5], state[9], state[13], state[1]
    state[2], state[6], state[10], state[14] = state[10], state[14], state[2], state[6]
    state[3], state[7], state[11], state[15] = state[15], state[3], state[7], state[11]
    return state


@classmethod
def _mix_columns(cls, state: list[int]) -> list[int]:
    return [
        cls.gal2[state[0]] ^ cls.gal3[state[1]] ^ state[2] ^ state[3],
        state[0] ^ cls.gal2[state[1]] ^ cls.gal3[state[2]] ^ state[3],
        state[0] ^ state[1] ^ cls.gal2[state[2]] ^ cls.gal3[state[3]],
        cls.gal3[state[0]] ^ state[1] ^ state[2] ^ cls.gal2[state[3]],
        cls.gal2[state[4]] ^ cls.gal3[state[5]] ^ state[6] ^ state[7],
        state[4] ^ cls.gal2[state[5]] ^ cls.gal3[state[6]] ^ state[7],
        state[4] ^ state[5] ^ cls.gal2[state[6]] ^ cls.gal3[state[7]],
        cls.gal3[state[4]] ^ state[5] ^ state[6] ^ cls.gal2[state[7]],
        cls.gal2[state[8]] ^ cls.gal3[state[9]] ^ state[10] ^ state[11],
        state[8] ^ cls.gal2[state[9]] ^ cls.gal3[state[10]] ^ state[11],
        state[8] ^ state[9] ^ cls.gal2[state[10]] ^ cls.gal3[state[11]],
        cls.gal3[state[8]] ^ state[9] ^ state[10] ^ cls.gal2[state[11]],
        cls.gal2[state[12]] ^ cls.gal3[state[13]] ^ state[14] ^ state[15],
        state[12] ^ cls.gal2[state[13]] ^ cls.gal3[state[14]] ^ state[15],
        state[12] ^ state[13] ^ cls.gal2[state[14]] ^ cls.gal3[state[15]],
        cls.gal3[state[12]] ^ state[13] ^ state[14] ^ cls.gal2[state[15]],
    ]